In [1]:
import sympy as sp
import numpy as np 
from numpy import radians
from time import perf_counter

In [2]:
num_poles = 2

pole_as = [0.1 for i in range(num_poles)]
pole_ls = [0.2 for i in range(num_poles)]
pole_ms = [0.05 for i in range(num_poles)]
pole_ds = [0.01 for i in range(num_poles)]
pole_Js = [0.1 for i in range(num_poles)]

m_c = 0.5
g = 9.81

In [3]:
t = sp.Symbol("t")
s = sp.Function("s")(t)
d_s = sp.diff(s, t)
dd_s = sp.diff(d_s, t)
thetas = [sp.Function(f"theta_{i+1}")(t) for i in range(num_poles)]
d_thetas = [sp.diff(theta,t) for theta in thetas]
dd_thetas = [sp.diff(d_theta,t) for d_theta in d_thetas]
tau = sp.Symbol("tau")

In [4]:
pole_pcs = []
for i, (theta, a) in enumerate(zip(thetas, pole_as)):
    prev_1 = 0
    prev_2 = 0
    for prev_l, prev_theta in list(zip(pole_ls, thetas))[:i]:
        prev_1 += -prev_l*sp.sin(prev_theta)
        prev_2 += prev_l*sp.cos(prev_theta)
    pole_pcs.append([s-a*sp.sin(theta)+prev_1, a*sp.cos(theta)+prev_2])

In [5]:
def square_sum(x: tuple):
    result = 0
    for x_i in x:
        result += x_i**2
    return result

T = 1/2*m_c*d_s**2
for m, pc, J, d_theta in zip(pole_ms, pole_pcs, pole_Js, d_thetas):
    d_pc = [sp.diff(p,t) for p in pc]
    T_p = 1/2*m*square_sum(d_pc) + 1/2*J*d_theta**2
    T += T_p
    
V = 0   
for m, pc in zip(pole_ms, pole_pcs  ):
    V += g*m*pc[1]
R = 0
prev_w = 0
for d, d_theta in zip(pole_ds, d_thetas):
    R += 1/2*d*(d_theta-prev_w)**2
    prev_w = d_theta

In [6]:
L = T-V
eqs = []
lh = sp.diff(sp.diff(L, d_s),t) - sp.diff(L, s) + sp.diff(R, d_s)
rh = tau
eqs.append(lh-rh)
for theta, d_theta in zip(thetas, d_thetas):
    L = T-V
    lh = sp.diff(sp.diff(L, d_theta),t) - sp.diff(L, theta) + sp.diff(R, d_theta)
    rh = 0
    eqs.append(lh-rh)

In [13]:
sols = sp.solve(eqs, dd_thetas + [tau])
for key, value in sols.items():
    sols[key] = sp.simplify(value)
    display(key, sols[key])

Derivative(theta_1(t), (t, 2))

(-0.00238101016965608*sin(theta_1(t) - 2*theta_2(t)) + 0.00975704473192398*sin(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t)**2 + 4.85425111040994e-5*sin(2*theta_1(t) - 2*theta_2(t))*Derivative(theta_1(t), t)**2 - 1.43336812213296*sin(theta_1(t)) + 0.000242712555520497*cos(theta_1(t) - 2*theta_2(t))*Derivative(s(t), (t, 2)) + 0.000970850222081988*cos(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t) - 0.000970850222081988*cos(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t) - 4.06575814682064e-20*cos(theta_1(t) + 2*theta_2(t))*Derivative(s(t), (t, 2)) - 0.146112958423339*cos(theta_1(t))*Derivative(s(t), (t, 2)) + 0.19514089463848*Derivative(theta_1(t), t) - 0.0975704473192398*Derivative(theta_2(t), t))/(4.85425111040994e-5*cos(2*theta_1(t) - 2*theta_2(t)) - 1.0000485425111)

Derivative(theta_2(t), (t, 2))

(-0.00995121477634038*sin(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t)**2 - 4.85425111040994e-5*sin(2*theta_1(t) - 2*theta_2(t))*Derivative(theta_2(t), t)**2 + 0.00714303050896823*sin(2*theta_1(t) - theta_2(t)) - 0.480964054270527*sin(theta_2(t)) - 0.00194170044416398*cos(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t) + 0.000970850222081989*cos(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t) + 0.000728137666561491*cos(2*theta_1(t) - theta_2(t))*Derivative(s(t), (t, 2)) - 0.0490279362151404*cos(theta_2(t))*Derivative(s(t), (t, 2)) - 0.0995121477634038*Derivative(theta_1(t), t) + 0.0995121477634038*Derivative(theta_2(t), t))/(4.85425111040994e-5*cos(2*theta_1(t) - 2*theta_2(t)) - 1.0000485425111)

tau

(2.41498992742895e-5*sin(theta_1(t) - 2*theta_2(t))*Derivative(theta_1(t), t)**2 - 7.29351229339094e-5*sin(2*theta_1(t) - theta_2(t))*Derivative(theta_2(t), t)**2 - 0.0149758501007257*sin(theta_1(t))*Derivative(theta_1(t), t)**2 + 0.0107324033397248*sin(2*theta_1(t)) - 0.00492706487706609*sin(theta_2(t))*Derivative(theta_2(t), t)**2 + 0.0011845525594039*sin(2*theta_2(t)) + 4.85425111040994e-6*cos(theta_1(t) - 2*theta_2(t))*Derivative(theta_1(t), t) - 2.42712555520497e-6*cos(theta_1(t) - 2*theta_2(t))*Derivative(theta_2(t), t) + 2.54848183296522e-5*cos(2*theta_1(t) - 2*theta_2(t))*Derivative(s(t), (t, 2)) - 7.28137666561491e-6*cos(2*theta_1(t) - theta_2(t))*Derivative(theta_1(t), t) + 7.28137666561491e-6*cos(2*theta_1(t) - theta_2(t))*Derivative(theta_2(t), t) - 0.00292225916846679*cos(theta_1(t))*Derivative(theta_1(t), t) + 0.00146112958423339*cos(theta_1(t))*Derivative(theta_2(t), t) + 0.00109402684400864*cos(2*theta_1(t))*Derivative(s(t), (t, 2)) + 0.000490279362151404*cos(theta_2(t)

In [8]:
str(sols[dd_thetas[0]])

'(-0.00238101016965608*sin(theta_1(t) - 2*theta_2(t)) + 0.00975704473192398*sin(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t)**2 + 4.85425111040994e-5*sin(2*theta_1(t) - 2*theta_2(t))*Derivative(theta_1(t), t)**2 - 1.43336812213296*sin(theta_1(t)) + 0.000242712555520497*cos(theta_1(t) - 2*theta_2(t))*Derivative(s(t), (t, 2)) + 0.000970850222081988*cos(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t) - 0.000970850222081988*cos(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t) - 4.06575814682064e-20*cos(theta_1(t) + 2*theta_2(t))*Derivative(s(t), (t, 2)) - 0.146112958423339*cos(theta_1(t))*Derivative(s(t), (t, 2)) + 0.19514089463848*Derivative(theta_1(t), t) - 0.0975704473192398*Derivative(theta_2(t), t))/(4.85425111040994e-5*cos(2*theta_1(t) - 2*theta_2(t)) - 1.0000485425111)'

In [9]:
sp.sympify(str(sols[dd_thetas[0]]))

(-0.00238101016965608*sin(theta_1(t) - 2*theta_2(t)) + 0.00975704473192398*sin(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t)**2 + 4.85425111040994e-5*sin(2*theta_1(t) - 2*theta_2(t))*Derivative(theta_1(t), t)**2 - 1.43336812213296*sin(theta_1(t)) + 0.000242712555520497*cos(theta_1(t) - 2*theta_2(t))*Derivative(s(t), (t, 2)) + 0.000970850222081988*cos(theta_1(t) - theta_2(t))*Derivative(theta_1(t), t) - 0.000970850222081988*cos(theta_1(t) - theta_2(t))*Derivative(theta_2(t), t) - 4.06575814682064e-20*cos(theta_1(t) + 2*theta_2(t))*Derivative(s(t), (t, 2)) - 0.146112958423339*cos(theta_1(t))*Derivative(s(t), (t, 2)) + 0.19514089463848*Derivative(theta_1(t), t) - 0.0975704473192398*Derivative(theta_2(t), t))/(4.85425111040994e-5*cos(2*theta_1(t) - 2*theta_2(t)) - 1.0000485425111)

In [10]:
def sympy2casadi(sympy_expr,sympy_var,casadi_var):
  import casadi
  assert casadi_var.is_vector()
  if casadi_var.shape[1]>1:
    casadi_var = casadi_var.T
  casadi_var = casadi.vertsplit(casadi_var)
  from sympy.utilities.lambdify import lambdify

  mapping = {'ImmutableDenseMatrix': casadi.blockcat,
             'MutableDenseMatrix': casadi.blockcat,
             'Abs':casadi.fabs
            }
  f = lambdify(sympy_var,sympy_expr,modules=[mapping, casadi])
  return f(*casadi_var)
 
import casadi as ca

sp_vars = [s,d_s,dd_s] + thetas + d_thetas
xy = sp.Matrix(sp_vars)

ca_vars= [ca.SX.sym("s"), ca.SX.sym("d_s"), ca.SX.sym("dd_s")]
for i in range(1,1+num_poles):
  ca_vars.append(ca.SX.sym("theta"+str(i)))
  ca_vars.append(ca.SX.sym("d_theta"+str(i)))
XY = ca.vertcat(*ca_vars)

print(sympy2casadi(sols[dd_thetas[1]],xy,XY))

@1=2, @2=4.85425e-05, @3=0.0995121, ((((((((((((-0.0490279*dd_s)*cos(d_theta1))+((0.000728138*dd_s)*cos(((@1*theta1)-d_theta1))))-((@2*sq(d_theta2))*sin(((@1*theta1)-(@1*d_theta1)))))+((0.00097085*d_theta2)*cos((theta1-d_theta1))))+(@3*d_theta2))-((0.00995121*sq(theta2))*sin((theta1-d_theta1))))-((0.0019417*theta2)*cos((theta1-d_theta1))))-(@3*theta2))-(0.480964*sin(d_theta1)))+(0.00714303*sin(((@1*theta1)-d_theta1))))/((@2*cos(((@1*theta1)-(@1*d_theta1))))-1.00005))


In [11]:
opti = ca.Opti()
opti_vars = opti.variable(2+2*num_poles+1)
print(sympy2casadi(sols[dd_thetas[0]],xy,opti_vars))

@1=vertsplit(opti0_x_1), @2=@1{2}, @3=@1{3}, @4=@1{4}, @5=@1{6}, @6=@1{5}, (((((((((((((-0.146113*@2)*cos(@3))+((0.000242713*@2)*cos(((2.*@4)-@3))))-((4.06576e-20*@2)*cos(((2.*@4)+@3))))-((0.00975704*sq(@5))*sin((@4-@3))))-((0.00097085*@5)*cos((@4-@3))))-(0.0975704*@5))-((4.85425e-05*sq(@6))*sin(((2.*@4)-(2.*@3)))))+((0.00097085*@6)*cos((@4-@3))))+(0.195141*@6))-(1.43337*sin(@3)))+(0.00238101*sin(((2.*@4)-@3))))/((4.85425e-05*cos(((2.*@4)-(2.*@3))))-1.00005))


In [12]:
values = np.array([10, 0, radians(0), 0, radians(0), 0, radians(0), 0] + [0])
variables = [s, d_s]
for theta, d_theta in zip(thetas, d_thetas):
    variables.append(theta)
    variables.append(d_theta)
variables.append(dd_s)

start_time = perf_counter()
# substitute the values of the variables into the solutions
sol_sub = sols.copy()
for i in range(100):
    for k, v in sols.items():
        a = v.subs({var: value for var, value in zip(variables, values)})
print(f"Time: {perf_counter()-start_time}")

Time: 0.193207599921152
